# Emotion Recognition Model

## We'll be using [Kaggle](https://www.kaggle.com/ashishbansal23/emotion-recognition/data) dataset to train our model

#### We'll start by importing the necessary libraries

In [41]:
import matplotlib as plt
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
%matplotlib inline

#### Let's break down the codes and understand it
- Sequential library basically means that we are going to create linear stack of layers where the output of neurons of one layer will serve as the input to the next layer and in this way we'll get something like tree with roots coming out and getting more complex, more info keras.
- Conv2D basically tries to fetch the important information from the image like curve, smoothness, or a particular patterns
- MaxPooling2D work is to reduce the size of the matrix without compromising with the information that it contains
- Flatten work is to convert the madrix to the one dimensional vector so that its each element could then be used as an input to the artificial neural network
- Dense is used to add more layer to the ann efficiently i suppose
- Batch Normalization, actually normalize the activation so that it is close to zero and the activation deviation is close to 1, more on [keras](https://keras.io/layers/normalization/)
- np_utils will help us in reformatting the dataset obtained from the kaggle

In [20]:
dataset = pd.read_csv('fer2013.csv')

- there is an interseting thing about this dataset, you won't be able to open it with the help of excel or google sheets, this is because the formatting of the dataset, I recommend using Spyder, to read this file
- the dataset has 3 columns emotion, pixels and Usage, we need to first extract the data whose Usage is for Training purposes and we'll then use it to train our model

<img src="spyder.png">

### Training data

In [21]:
train_dataset = dataset[["emotion", "pixels"]][dataset["Usage"]=="Training"]
train_dataset['pixels'] = train_dataset['pixels'].apply(lambda im: np.fromstring(im, sep=' '))
X_train = np.vstack(train_dataset['pixels'].values)
y_train = np.vstack(train_dataset['emotion'])

- first we are extracting the data correspoding to the Usage 'Training'
- because the data in the Pixels column is in string we need to convert it into array, because our model won't recognise input data as string
- np.vstack is basically used to combine all the information into array
- in y_train we want the labels whether the information is of happy,sad,angry etc

### Test data

In [22]:
test_dataset = dataset[["emotion", "pixels"]][dataset["Usage"]=="PublicTest"]
test_dataset['pixels'] = test_dataset['pixels'].apply(lambda im: np.fromstring(im, sep=' '))
X_test = np.vstack(test_dataset['pixels'].values)
y_test = np.vstack(test_dataset['emotion'])

### Reshaping the Matrix

In [23]:
X_train = X_train.reshape(-1, 48, 48, 1)
X_test = X_test.reshape(-1, 48, 48, 1)

- we are reshaping the matrix because the our model wants a matrix having dimension (-1, 48, 48, 1)
- we have 2034 pixel data in one row in pixel columns so it can be reshaped into 48 * 48 matrix

In [24]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

- as there are 7 categories in the column emotion and each category is represented by the number 1 to 7, but as we know these are labels so we need to create dummy variables to make our model don't interpret these labels as numbers like, 7 < 6, because these are not numbers, they are labels. 

### Convolution

''' applying the convolution
 actually we are providing the pixels to the model that was given
 so we don't need to convert the image to pixel
 but if we want to test the image then we have 
to manually convert it into the 48 * 48 size further it should be greyscale
'''

In [38]:
classifier = Sequential()

classifier.add(Conv2D(64, (5, 5), input_shape=(48,48,1), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (5, 5), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(32, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.5))

classifier.add(Conv2D(32, (3, 3)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.5))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

### Full Connection

In [39]:
classifier.add(Flatten())

classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=7, activation='softmax'))

classifier.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
batch_normalization_8 (Batch (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 18, 18, 64)        102464    
_________________________________________________________________
batch_normalization_9 (Batch (None, 18, 18, 64)        256       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 7, 7, 32)          18464     
__________

### Save the best model

In [42]:
checkpointer = ModelCheckpoint(filepath='highly_trained_model.h5', verbose=1, save_best_only=True)

### Define the number of Epochs

In [43]:
epochs = 15

### Train the Model

In [ ]:
classifier.fit(X_train, y_train,
                      epochs=epochs,
                      shuffle=True,
                      batch_size=32,
                      validation_data=(X_test,y_test),
                      callbacks=[checkpointer],
                      verbose=2)

### Testing the Model on a Different Dataset provided in the dataset

In [ ]:
# test the model
test_f = dataset[["emotion", "pixels"]][dataset["Usage"] == "PrivateTest"]
test_f["pixels"] = test_f["pixels"].apply(lambda im: np.fromstring(im, sep=' '))

x_test_private = np.vstack(test["pixels"].values)
y_test_private = np.array(test["emotion"])

x_test_private = x_test_private.reshape(-1, 48, 48, 1)
y_test_private = np_utils.to_categorical(y_test_private)

score = classifier.evaluate(x_test_private, y_test_private, verbose=0)